In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import numpy as np
from pyspark.sql.window import Window

list=[[790026,9160,0,1,0,26.1184,23.2954],
[790026,13509,0,0,1,np.inf,-np.inf],
[790026,9162,0,0,0,25.03535,23.48585],
[790026,13510,0,0,1,np.inf,-np.inf],
[790048,9162,0,0,0,33.5,30.5],
[790048,13509,0,0,1,np.inf,-np.inf],
[790048,13510,0,0,0,np.NaN,np.NaN],
[790048,9160,0,1,0,33.94075,30.75925 ]]
df= spark.createDataFrame(list,['SEQ_ID','TOOL_ID','isfleetlevel','is_golden_limit','use_golden_limit','New_UL','New_LL'])
df.show()
w=Window().partitionBy("SEQ_ID").orderBy("SEQ_ID")
df1=df.select(F.col("is_golden_limit").alias("use_golden_limit"),F.col("New_UL").alias("New_UL1"),F.col("New_LL").alias("New_LL1"),"SEQ_ID").filter(F.col("is_golden_limit")==1).withColumn('row_num',F.row_number().over(w)).filter(F.col("row_num")==1).drop("row_num")
          
          




df1.show()

+------+-------+------------+---------------+----------------+--------+---------+
SEQ_ID|TOOL_ID|isfleetlevel|is_golden_limit|use_golden_limit| New_UL| New_LL|
+------+-------+------------+---------------+----------------+--------+---------+
790026| 9160| 0| 1| 0| 26.1184| 23.2954|
790026| 13509| 0| 0| 1|Infinity|-Infinity|
790026| 9162| 0| 0| 0|25.03535| 23.48585|
790026| 13510| 0| 0| 1|Infinity|-Infinity|
790048| 9162| 0| 0| 0| 33.5| 30.5|
790048| 13509| 0| 0| 1|Infinity|-Infinity|
790048| 13510| 0| 0| 0| NaN| NaN|
790048| 9160| 0| 1| 0|33.94075| 30.75925|
+------+-------+------------+---------------+----------------+--------+---------+

+----------------+--------+--------+------+
use_golden_limit| New_UL1| New_LL1|SEQ_ID|
+----------------+--------+--------+------+
 1| 26.1184| 23.2954|790026|
 1|33.94075|30.75925|790048|
+----------------+--------+--------+------+

In [2]:
df2=df.join(df1.hint("broadcast"), on=['use_golden_limit','SEQ_ID'], how='left')
df3=df2.withColumn("New_UL_Final", F.when((F.col("use_golden_limit")==1),F.col("New_UL1")).otherwise(F.col("New_UL")))\
   .withColumn("New_LL_Final", F.when((F.col("use_golden_limit")==1),F.col("New_LL1")).otherwise(F.col("New_LL")))\
   .orderBy("SEQ_ID").drop("New_UL","New_LL","New_LL1","New_UL1")
df4=df3.select("SEQ_ID","TOOL_ID","isfleetlevel","is_golden_limit","use_golden_limit",F.col("New_UL_Final").alias("New_UL"),
          F.col("New_LL_Final").alias("New_LL"))
df4.show()

+------+-------+------------+---------------+----------------+--------+--------+
SEQ_ID|TOOL_ID|isfleetlevel|is_golden_limit|use_golden_limit| New_UL| New_LL|
+------+-------+------------+---------------+----------------+--------+--------+
790026| 9162| 0| 0| 0|25.03535|23.48585|
790026| 13509| 0| 0| 1| 26.1184| 23.2954|
790026| 13510| 0| 0| 1| 26.1184| 23.2954|
790026| 9160| 0| 1| 0| 26.1184| 23.2954|
790048| 13510| 0| 0| 0| NaN| NaN|
790048| 9162| 0| 0| 0| 33.5| 30.5|
790048| 13509| 0| 0| 1|33.94075|30.75925|
790048| 9160| 0| 1| 0|33.94075|30.75925|
+------+-------+------------+---------------+----------------+--------+--------+